# `geom_curve()` on Map


In [1]:
import pandas as pd
from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def set_stay_time(data):
    current_moves = moves_df[(moves_df.departure == data['name'])|(moves_df.arrival == data['name'])]
    years = current_moves.year.sort_values().unique()
    years_combined = []
    for year in years:
        if len(years_combined) == 0 or years_combined[-1][1] + 1 != year:
            years_combined.append((year, year))
        else:
            years_combined[-1] = (years_combined[-1][0], year)
    years_combined = [str(year_from) if year_from == year_to else '{0}-{1}'.format(year_from, year_to) \
                      for year_from, year_to in years_combined]
    data['years'] = ', '.join(years_combined)
    return data

places_df = pd.read_csv('https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/places.csv')
moves_df = pd.read_csv('https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pushkin/moves.csv')
df = moves_df.merge(places_df, left_on='departure', right_on='name')\
             .rename(columns=dict(longitude='from_lon', latitude='from_lat'))\
             .drop(columns=['name'])\
             .merge(places_df, left_on='arrival', right_on='name')\
             .rename(columns=dict(longitude='to_lon', latitude='to_lat'))\
             .drop(columns=['name'])
places_df['years'] = ''
places_df = places_df.apply(set_stay_time, axis=1)

In [4]:
ggplot() \
    + geom_livemap(const_size_zoomin=3) \
    + geom_curve(aes(x='from_lon', y='from_lat', xend='to_lon', yend='to_lat', color='path'), data=df,
                 size_start=3, size_end=3,
                 spacer=2.5,
                 size=0.5,
                 curvature=0.2,
                 arrow=arrow(type='closed', length=8, angle=15),
                 tooltips='none') \
    + geom_point(aes(x='longitude', y='latitude'), data=places_df, 
                 size=3, shape=21, color='black', fill='white', alpha=0.75,
                 tooltips=layer_tooltips().title('@name').line('visited in|@years')) \
    + scale_color_manual(name='trip name', values=['#addd8e', '#e34a33', '#8856a7', '#2c7fb8',
                                                   '#1c9099', '#006d2c', '#fec44f', '#636363']) \
    + coord_cartesian(xlim=[26, 58], ylim=[38, 62]) \
    + ggtitle("Alexander Pushkin's Trips") \
    + ggsize(1000, 800) \
    + theme_void() \
    + theme(legend_position=[1, 1], legend_justification=[1, 1], legend_title="blank")